In [1]:
import re
import os
import sys
import json
import random
from importlib import reload

from bart2query.sparql.predict import whether_equal, post_process
from tqdm import tqdm
from bart2query.sparql.sparql_engine import get_sparql_answer

random.seed(666)

In [ ]:
# new data
train_data = json.load(open("./dataset_new/train.json"))
train_sparql = [item['sparql'] for item in train_data]
train_program = [item['program'] for item in train_data]

In [ ]:
test_data = json.load(open("./dataset_new/test.json"))
test_sparql = [item['sparql'] for item in test_data]
test_program = [item['program'] for item in test_data]

In [2]:
def gen_ir(ir):
    for token in ["<E>","</E>","<ES>","</ES>","<A>","</A>","<R>","</R>","<V>","</V>","<Q>","</Q>","<C>","</C>"]:
        ir = ir.replace(" {}".format(token), token)
        ir = ir.replace("{} ".format(token), token)
    return ir

In [3]:
import KqaPro_Parser.ir.SparqlTranslator
from KqaPro_Parser.ir.SparqlTranslator import Sparql_translator
sparql_translator = Sparql_translator()

In [6]:
ir = '''what is<ES><E>entity1</E>whose<A>attr1</A>is larger than<A>attr2</A>of<E>entity2</E></ES>'''
sparql_translator.ir_to_sparql(gen_ir(ir))

line 1:45 no viable alternative at input 'whose<A>attr1</A>islarger than'


'SELECT DISTINCT ?e WHERE { ?e <pred:name> "entity1" . ?e <> ?pv . ?pv <pred:value> "" .  }'

In [ ]:
from utils.load_kb import DataForSPARQL
from utils.data import DataLoader

# input_dir = "./exp_files_new/UIR/full/end2end/sparql/"
# vocab_json = os.path.join(input_dir, 'vocab.json')
# data = os.path.join(input_dir, 'train.pt')
# data = DataLoader(vocab_json, data, 128)
kb = DataForSPARQL(os.path.join("./dataset_new/", 'kb.json'))

In [ ]:
pred_ir = []
total, correct = 0, 0
with open("./ir_results.txt") as f:
    for line in f:
        line = line.strip()
        pred_ir.append(gen_ir(line))


In [ ]:
from IR_unified.self_correct import IRCorrector
ir_corrector = IRCorrector()
# corrected_pred_ir = [ir_corrector.self_correct(ir) for ir in pred_ir]

In [ ]:
pred_sparql = []
correct_pred_sparql = []
grammar_wrong_irs = []
for ir, i in zip(pred_ir, test_data):
    pred_sparql.append(sparql_translator.ir_to_sparql(ir))
    
    correct_ir = ir_corrector.self_correct(ir)
    correct_pred_sparql.append(sparql_translator.ir_to_sparql(correct_ir))

In [ ]:
total, correct = 0, 0
true_false = []
false_true = []
false_false = []

for pred, gold, correct_pred in tqdm(zip(pred_sparql, test_sparql, correct_pred_sparql)):
    pred_res = get_sparql_answer(pred, kb)
    gold_res = get_sparql_answer(gold, kb)
    is_match = whether_equal(gold_res, pred_res)

    correct_pred_res = get_sparql_answer(correct_pred, kb)
    correct_match = whether_equal(gold_res, correct_pred_res)

    if is_match:
        if correct_match:
            correct += 1
        else:
            true_false.append((pred, gold, correct_pred))
    else:
        if correct_match:
            false_true.append((pred, gold, correct_pred))
        else:
            false_false.append((pred, gold, correct_pred))
            
    total += 1
print("IR: {}/{}".format(correct, total))

In [ ]:
to_fixed = []
for i in false_false:
    to_fixed.append(list(i).append(test_data[test_sparql.index(i[1])]["rewrite"]))

In [ ]:
sum([int(pred != correct_pred) for pred, correct_pred in zip(pred_sparql, correct_pred_sparql)])

In [ ]:
len(true_false), len(false_true), len(false_false)

In [ ]:
import KqaPro_Parser.program_v2.ProgramIRTranslator
reload(KqaPro_Parser.program_v2.ProgramIRTranslator)
from KqaPro_Parser.program_v2.ProgramIRTranslator import IR_translator
ir_translator = IR_translator()

def gen_ir(i):
    ir = ir_translator.program_to_ir(i)
    for token in ["<E>","</E>","<ES>","</ES>","<A>","</A>","<R>","</R>","<V>","</V>","<Q>","</Q>","<C>","</C>"]:
        ir = ir.replace(" {}".format(token), token)
        ir = ir.replace("{} ".format(token), token)
    return ir

In [ ]:
ir_list = []
for point in train_program:
    ir = gen_ir(point)
    ir_list.append(ir)
    

In [ ]:
count, correct = 0, 0
all_answers = []
for batch in tqdm(data, total=len(data)):
    source_ids, source_mask, choices, target_ids, answer = [x.to('cpu') for x in batch]
    all_answers.extend(answer.cpu().numpy())
    
given_answer = [data.vocab['answer_idx_to_token'][a] for a in all_answers]

In [ ]:
assert len(pred_list) == len(given_answer)
len(pred_list), len(given_answer)

In [ ]:
count, correct = 0, 0
wrong_list = []
for s, ir, d in tqdm(zip(pred_list, ir_list, train_data)):
    pred_answer = get_sparql_answer(s, kb)
    gold_answer = get_sparql_answer(d["sparql"], kb)

    is_match = whether_equal(gold_answer, pred_answer)
    if is_match:
        correct += 1
    else:
        wrong_list.append([d["sparql"], s, ir, d["origin"], d["program"]])

    count += 1

acc = correct / count
print('acc: {}'.format(acc))

In [ ]:
count, correct 

In [ ]:
for i in wrong_list:
    print(i[0])
    print(i[1])
    print(i[2])
    print(i[3])
    print('\n')

In [ ]:
for a, s in tqdm(zip(given_answer, pred_list)):
    pred_answer = get_sparql_answer(s, kb)
    
    is_match = whether_equal(a, pred_answer)
    if is_match:
        correct += 1

    count += 1

acc = correct / count
print('acc: {}'.format(acc))